In [1]:
import pandas as pd
import numpy as np
from neighborhoods import admin_wards
from neighborhoods import district_neighborhoods

In [2]:
df = pd.read_json('../data/raw/scraped_data.json')
df = df[['Price', 'Title', 'Full description', 'Bedrooms', 'Features', 'Area']]

In [3]:
df['Price'] = df['Price'].str.replace('.', '').str.replace('€', '').apply(lambda x: int(x))
df['price'] = df['Price'].copy()
df['Title'] = df['Title'].str.replace(',', '')
df['Title'] = df['Title'].apply(lambda x: x.split())
df['neighborhood'] = df['Title'].apply(lambda x: x[-2])

/var/folders/zm/z5q67jgd0vv23h_ypgdpl9ph0000gn/T/ipykernel_32260/3288621376.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Price'] = df['Price'].str.replace('.', '').str.replace('€', '').apply(lambda x: int(x))


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Price             1284 non-null   int64 
 1   Title             1284 non-null   object
 2   Full description  1283 non-null   object
 3   Bedrooms          1284 non-null   object
 4   Features          1284 non-null   object
 5   Area              1284 non-null   object
 6   price             1284 non-null   int64 
 7   neighborhood      1284 non-null   object
dtypes: int64(2), object(6)
memory usage: 80.4+ KB


In [5]:
df_area_values = []
for index, item in enumerate(df['Features'].values):
    found = False
    for value in item:
        if 'construidos' in value:
            df_area_values.append(value)
            found = True
            break
    if not found:
        df_area_values.append(df['Area'].iloc[index])

df['m2'] = df_area_values

df['m2'] = df['m2'].str.replace('.', '')
df['m2'] = df['m2'].apply(lambda x: float(x.split()[0]))

/var/folders/zm/z5q67jgd0vv23h_ypgdpl9ph0000gn/T/ipykernel_32260/1803489140.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['m2'] = df['m2'].str.replace('.', '')


In [6]:
df_rooms_values = []

for index, item in enumerate(df['Features'].values):
    found = False
    for value in item:
        if 'habitaciones' in value or 'habitación' in value:
            df_rooms_values.append(value)
            found = True
            break
    if not found:
        df_rooms_values.append(df['Bedrooms'].iloc[index])

df['rooms'] = df_rooms_values
df['rooms'] = df['rooms'].apply(lambda x: x.split()[0]).replace("Sin", '0').apply(lambda x: int(x))

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Price             1284 non-null   int64  
 1   Title             1284 non-null   object 
 2   Full description  1283 non-null   object 
 3   Bedrooms          1284 non-null   object 
 4   Features          1284 non-null   object 
 5   Area              1284 non-null   object 
 6   price             1284 non-null   int64  
 7   neighborhood      1284 non-null   object 
 8   m2                1284 non-null   float64
 9   rooms             1284 non-null   int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 100.4+ KB


In [8]:
df_house_type_2 = []

for index, item in enumerate(df['Features'].values):
    found = False
    for value in item:
        if 'exterior' in value.lower():
            df_house_type_2.append('exterior')
            found = True
            break
        if 'interior' in value.lower():
            df_house_type_2.append('interior')
            found = True
            break
    if not found:
        df_house_type_2.append('not indicated')

df['house_type_2'] = df_house_type_2


In [9]:
df_house_type = []

for index, item in enumerate(df['Features'].values):
    found = False
    for value in item:
        if value.lower().startswith('planta'):
            df_house_type.append(value)
            found = True
            break
        if 'bajo' in value.lower():
            df_house_type.append('bajo')
            found = True
            break
        if 'entreplanta' in value.lower():
            df_house_type.append('entreplanta')
            found = True
            break
        if 'sotano' in value.lower():
            df_house_type.append('chalet')
            found = True
            break
        if 'semisotano' in value.lower():
            df_house_type.append('semi-sotano')
            found = True
            break
        if 'casa' in value.lower():
            df_house_type.append('casa')
            found = True
            break
        if 'chalet' in value.lower():
            df_house_type.append('chalet')
            found = True
            break
    if not found:
        full_desc = df['Full description'].iloc[index]
        if 'sotano' in full_desc:
            df_house_type.append('sotano')
            found = True
        elif 'semisotano' in full_desc:
            df_house_type.append('semi-sotano')
            found = True
        elif 'casa' in full_desc:
            df_house_type.append('casa')
            found = True
        elif 'chalet' in full_desc:
            df_house_type.append('chalet')
            found = True
        else:
            df_house_type.append('not indicated')

df['house_type'] = df_house_type
df['house_type'] = df['house_type'].str.replace(' exterior', "").str.replace(' interior', "").str.replace('ª', "").str.replace('Planta', "planta")

In [10]:
df_garage = []

for index, item in enumerate(df['Features'].values):
    found = False
    for value in item:
        if 'garaje' in value.lower():
            df_garage.append(1)
            found = True
            break
    if not found:
        df_garage.append(0)

df['garage'] = df_garage

In [11]:
df_elevator = []

for index, item in enumerate(df['Features'].values):
    found = False
    for value in item:
        if 'ascensor' in value.lower():
            df_elevator.append(1)
            found = True
            break
    if not found:
        df_elevator.append(0)

df['elevator'] = df_elevator

In [12]:
df['neighborhood'] = df['neighborhood'].str.replace("-", " ")

In [13]:
df_neighborhoods = set(df['neighborhood'].values)

for item in admin_wards:
    for barrio in df_neighborhoods:
        if item in barrio:
            df['neighborhood'] = df['neighborhood'].replace(barrio, item)
        elif barrio in item:
            df['neighborhood'] = df['neighborhood'].replace(barrio, item)

In [14]:
replace_tuples = [('Fresno', 'Mirasierra'),
                  ('Tablas', 'Valverde') ,
                  ('Montecarmelo', "Mirasierra"),
                  ("Manoteras", 'Apóstol Santiago'),
                  ('Gavia', 'Ensanche de Vallecas'),
                  ('Sanchinarro', 'Valdefuentes'), 
                  ('Ambroz', "Casco Histórico de Vicálvaro"),
                  ('Frontera', 'Palos de Moguer')
]

In [15]:
for item in replace_tuples:
    df['neighborhood'] = df['neighborhood'].replace(item[0], item[1])

In [16]:
df = df[df['neighborhood']!= 'Carabanchel']

In [17]:
df['district'] = df['neighborhood'].copy()

In [18]:
df_district_unique = set(df['district'].values)

for item in df_district_unique:
    for key in district_neighborhoods:
        if item in district_neighborhoods[key]:
            df['district'] = df['district'].replace(item, key)

In [19]:
df = df[['price', 'house_type', 'house_type_2', 'rooms', 'm2', 'elevator',
       'garage', 'neighborhood', 'district']]

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1282 entries, 0 to 1283
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         1282 non-null   int64  
 1   house_type    1282 non-null   object 
 2   house_type_2  1282 non-null   object 
 3   rooms         1282 non-null   int64  
 4   m2            1282 non-null   float64
 5   elevator      1282 non-null   int64  
 6   garage        1282 non-null   int64  
 7   neighborhood  1282 non-null   object 
 8   district      1282 non-null   object 
dtypes: float64(1), int64(4), object(4)
memory usage: 100.2+ KB


In [22]:
df = df[~df.apply(lambda x: x.astype(str).str.contains('not indicated')).any(axis=1)]
df.shape

(1066, 9)

In [24]:
df.to_csv('../data/processed/scraped_data_cleaned_Emanuela.csv')